# TesiSquare Dataset Analysis

In [8]:
#We include all the libraries for the code
import pandas as pd  #2.2.3 Version
import numpy as np  #2.2.1 Version
import matplotlib.pyplot as plt  #3.10.0 Version
import seaborn as sns  #0.13.2 Version
import warnings  
from sklearn.impute import SimpleImputer  #1.6.1 Version
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

In [9]:
dataset = pd.read_csv("delivery_data.csv") #Opening the .csv file
dataset.info() #We read general info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14554 entries, 0 to 14553
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            14554 non-null  int64  
 1   SERVICETYPE           14340 non-null  object 
 2   VEHICLETYPE           14331 non-null  object 
 3   DEPARTURE_COUNTRY     14554 non-null  object 
 4   DEPARTURE_ZIPCODE     14548 non-null  float64
 5   ARRIVAL_COUNTRY       14554 non-null  object 
 6   ARRIVAL_ZIPCODE       14554 non-null  int64  
 7   SHIPPING_DATE         14554 non-null  object 
 8   GROSS_WEIGHT_KG       14554 non-null  float64
 9   NET_WEIGHT_KG         14554 non-null  float64
 10  VOLUME_M3             14554 non-null  float64
 11  DECLARED_DISTANCE_KM  11577 non-null  float64
 12  ACTUAL_DELIVERY_DATE  14554 non-null  object 
 13  DELIVERY_TIME_HH      14554 non-null  int64  
 14  WDAY                  14554 non-null  int64  
dtypes: float64(5), int6

In [10]:
dataset.describe() #Description of the dataset

,Unnamed: 0,DEPARTURE_ZIPCODE,ARRIVAL_ZIPCODE,GROSS_WEIGHT_KG,NET_WEIGHT_KG,VOLUME_M3,DECLARED_DISTANCE_KM,DELIVERY_TIME_HH,WDAY
count,14554.000000,14548.000000,14554.000000,14554.000000,14554.000000,14554.000000,11577.000000,14554.000000,14554.000000
mean,7277.500000,53013.321694,41699.581352,9.744264,8.002172,0.188305,352.646287,75.783633,1.730177
std,4201.522244,22512.124864,30222.758997,34.946037,31.718243,0.983377,166.566613,79.110044,1.507748
min,1.000000,12.000000,10.000000,0.020000,0.000000,0.000000,0.640000,-1344.000000,0.000000
25%,3639.250000,56122.750000,17031.000000,1.000000,0.000000,0.010000,223.850000,37.000000,0.000000
50%,7277.500000,63076.000000,35042.000000,1.370000,0.000000,0.010000,353.190000,71.000000,1.000000
75%,10915.750000,63076.000000,66020.000000,3.160000,2.280000,0.110000,468.690000,108.000000,3.000000
max,14554.000000,99208.000000,99518.000000,1039.820000,948.840000,105.000000,1090.610000,1920.000000,6.000000


## Cleaned Dataset

#### We decide to create a clean dataset (without any missing data) in order to have more accurate results

In [11]:

#Map the 'VEHICLETYPE' column values to corresponding numeric codes.
dataset['VEHICLETYPE'] = dataset['VEHICLETYPE'].map({
    'Standard': 0,
    'Express': 1,
    'Ground' : 2,
    'Express 2nd Day' : 3,
    'Express Next Day' : 4,
    'Economy Express' : 5   
})

#Map the 'SERVICETYPE' column values to corresponding numeric codes.
dataset['SERVICETYPE'] = dataset['SERVICETYPE'].map({  # We associate a number with each type of vehicle
    'E-commerce': 0,
    'Corriere espresso': 1,
    'Via gomma' : 2,
    'E-commerce Resi' : 3,
    'Corriere espresso resi' : 4
})

#SimpleImputer is used to fill missing values with the most frequent value (mode) of each column.
categorical_cols = ['SERVICETYPE', 'VEHICLETYPE']
categorical_imputer = SimpleImputer(strategy='most_frequent')
dataset[categorical_cols] = categorical_imputer.fit_transform(dataset[categorical_cols])

# Replace missing values (NaN) in the 'DEPARTURE_ZIPCODE' column with 0, 
# then convert the entire column to integer type.
dataset = dataset[(dataset['VEHICLETYPE'] != "Ore 10/10:30") & (dataset['VEHICLETYPE'] != "Documents") & (dataset['VEHICLETYPE'] != "Ore 12")] 
dataset['DEPARTURE_ZIPCODE'] = dataset['DEPARTURE_ZIPCODE'].fillna(0).astype(int)
dataset['SERVICETYPE'] = dataset['SERVICETYPE'].astype(int)
dataset['VEHICLETYPE'] = dataset['VEHICLETYPE'].astype(int)

dataset = dataset[dataset['DELIVERY_TIME_HH'] > 0]

print(dataset) #test


       Unnamed: 0  SERVICETYPE  VEHICLETYPE DEPARTURE_COUNTRY  \
24             25            1            1                IT   
27             28            2            0                IT   
28             29            2            0                IT   
29             30            2            0                IT   
30             31            2            0                IT   
...           ...          ...          ...               ...   
14549       14550            3            2                US   
14550       14551            3            2                US   
14551       14552            3            0                IT   
14552       14553            3            2                US   
14553       14554            3            2                US   

       DEPARTURE_ZIPCODE ARRIVAL_COUNTRY  ARRIVAL_ZIPCODE  \
24                 63811              IT            26100   
27                 63076              IT            25020   
28                 63076            

#### We also decided to remove any row with negative or zero delivery time

In [12]:
#Save the cleaned DataFrame to a CSV file without including the index.
dataset = dataset[dataset['DELIVERY_TIME_HH'] > 0]
print(dataset['DELIVERY_TIME_HH'].head())
#dataset.to_csv('cleaned_dataset.csv', index=False)

24    193
27     48
28     48
29     48
30     48
Name: DELIVERY_TIME_HH, dtype: int64


In [13]:
#Print the data types of each column in the dataset
print(dataset.dtypes)

Unnamed: 0                int64
SERVICETYPE               int64
VEHICLETYPE               int64
DEPARTURE_COUNTRY        object
DEPARTURE_ZIPCODE         int64
ARRIVAL_COUNTRY          object
ARRIVAL_ZIPCODE           int64
SHIPPING_DATE            object
GROSS_WEIGHT_KG         float64
NET_WEIGHT_KG           float64
VOLUME_M3               float64
DECLARED_DISTANCE_KM    float64
ACTUAL_DELIVERY_DATE     object
DELIVERY_TIME_HH          int64
WDAY                      int64
dtype: object
